In [0]:
#!import "../DataModel/DataStructure"
using System.Linq.Expressions;
using static Systemorph.Vertex.Equality.IdentityPropertyExtensions;

In [0]:
public class IdentityPropertyReader<T> where T : BaseVariableIdentity {
    private Func<T, string> identityReader;
    private string[] propertiesExcluded;
    
    private IdentityPropertyReader(string[] props) {
        propertiesExcluded = props;
        identityReader = GetIdentityReader();
    }

    private static Dictionary<string, IdentityPropertyReader<T>> InstancesByExcludedProperties = new();

    private Func<T, string> GetIdentityReader() {
        var parameter = Expression.Parameter(typeof(T));
        var expression = typeof(T).GetIdentityProperties().Where(x => !propertiesExcluded.Contains(x.Name))
                                  .SelectMany(x => new Expression[]{ Expression.Constant(x.Name.ToString()+":"),
                                                   Expression.Call(typeof(IdentityPropertyReader<T>).GetMethod(nameof(GetString)), Expression.Convert(Expression.Property(parameter, x.Name), typeof(Object))) }
                                  ).Aggregate((x, y) => Expression.Call(typeof(IdentityPropertyReader<T>).GetMethod(nameof(IdentityPropertyReader<RawVariable>.Concat)), x, y) );
        return Expression.Lambda<Func<T,string>>(expression, parameter).Compile();
    }

    public static IdentityPropertyReader<T> GetInstance(params string[] excludingProperties) {
        var key = string.Join(",", excludingProperties.OrderBy(x=>x));
        if(!InstancesByExcludedProperties.TryGetValue(key, out var instance))
            return InstancesByExcludedProperties[key] = new IdentityPropertyReader<T>(excludingProperties);
        return instance;
    }
    public string GetIdentityString(T x) => identityReader(x);
    
    public static string Concat(string first, string second) => first + " " + second;
    public static string GetString(Object item) => (item == null) ? "" : item.ToString();
}